# In this notebook, we will search some twitte message

# Pre-requise: Create twitter developer account and generate api credentials
Step 1: Apply for a Twitter Developer Account

Go to the Twitter developer site to apply for a developer account. Here, you have to select the Twitter user responsible for this account. It should probably be you or your organization. Here’s what this page looks like:



In [2]:
import tweepy as tw
import json
import pandas as pd
import pyarrow.parquet as pq
import s3fs
from pyarrow import fs
import pyarrow as pa
import os
from datetime import date

# Download tweets
## Step1. Setup the twitter api credential


In [3]:
consumer_key = "9F2cDP6mBO001MJtFyLybWGqT"
consumer_secret = "czoiPDtFaNrUboAQEPPXpb9EMwbTgW7WB6bZYxpBljy2Ozqrqc"
access_token = "207529473-0NYW8AbMELgWLUOtB8zCLYKSH2T3BPzb4iqr15Df"
access_token_secret = "TsIFoTd0yILe2pIjDczFf56ocDzxoQfaGxnDS10asw8qz"

## Step2. Create an instance of the twitter client

In [4]:
client_auth = tw.OAuthHandler(consumer_key, consumer_secret)
client_auth.set_access_token(access_token, access_token_secret)
api = tw.API(client_auth, wait_on_rate_limit=True, retry_count=5, retry_delay=1)

In [5]:
try:
    api.verify_credentials()
    print("Authentication OK")
except Exception as e:
    print("Error during authentication")

Authentication OK


## Step3. Get tweets that you are intrested in 

We have created the twitter client, now we are ready to search the tweets messages. Here we use the search_tweets to find all the tweets that contains certain key words of certain language. We don't use date to filter tweets because the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
For more details about the search_tweets
https://docs.tweepy.org/en/stable/api.html?highlight=search%20tweet#tweepy.API.search_tweets

If you want to search older tweets, you can use the search_full_archive method.
https://docs.tweepy.org/en/stable/api.html?highlight=search%20tweet#tweepy.API.search_full_archive



### Configure the search filter

In [9]:
# filter the search result by using below key words 
search_words = "#insee"
# We can get tweet before certain date
# until_date = "2021-11-24
# specify the 
language="fr"
# the max tweet number will be retained in the result
max_tweet_count=1000000

In [10]:
# Get the tweets
tweets = api.search_tweets(q=search_words,lang="fr",result_type="mixed", count=max_tweet_count)

# tweets is a list of object status, which has an attribute _json which is the actual tweet in json string.
print(tweets[0]._json)


{'created_at': 'Tue Nov 30 12:55:51 +0000 2021', 'id': 1465665879191048193, 'id_str': '1465665879191048193', 'text': "L'#INSEE affirme que le #PouvoirDAchat des Français augmente en 2021😳\nhttps://t.co/pFDxmQZSds\nÉvidemment, avec une… https://t.co/eQqv0L7VED", 'truncated': True, 'entities': {'hashtags': [{'text': 'INSEE', 'indices': [2, 8]}, {'text': 'PouvoirDAchat', 'indices': [24, 38]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/pFDxmQZSds', 'expanded_url': 'https://www.insee.fr/fr/statistiques/5897131', 'display_url': 'insee.fr/fr/statistique…', 'indices': [70, 93]}, {'url': 'https://t.co/eQqv0L7VED', 'expanded_url': 'https://twitter.com/i/web/status/1465665879191048193', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [116, 139]}]}, 'metadata': {'result_type': 'popular', 'iso_language_code': 'fr'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 

In [11]:
# For example, if we want to get the name of the sender, date and text of a tweet

tweet_dict=tweets[0]._json
text=tweet_dict.get("text")
user_name=tweet_dict.get("user").get("name")
date=tweet_dict.get("created_at")
print(f"name:{user_name} | message: {text} | date:{date}")

name:Philippe Herlin | message: L'#INSEE affirme que le #PouvoirDAchat des Français augmente en 2021😳
https://t.co/pFDxmQZSds
Évidemment, avec une… https://t.co/eQqv0L7VED | date:Tue Nov 30 12:55:51 +0000 2021


## Step4. Generate a data frame 

Now we have the tweet, we want to generate a dataframe based on the tweet 

In [12]:

def generate_tweet_df(tweets):
     # init dataframe
    df = pd.DataFrame(columns=['name','date','text'])
    index=0
    for tweet in tweets:
        # get column value for each tweet
        tweet_dict=tweet._json
        text=tweet_dict.get("text")
        user_name=tweet_dict.get("user").get("name")
        date=tweet_dict.get("created_at")
        # add new row to the dataframe
        df.loc[index] = pd.Series({'name':user_name, 'date':date, 'text':text})
        index=index+1
    return df
       
        

In [13]:
df=generate_tweet_df(tweets)

In [14]:
df.head(10)

,name,date,text
0,Philippe Herlin,Tue Nov 30 12:55:51 +0000 2021,L'#INSEE affirme que le #PouvoirDAchat des Fra...
1,"Richard Eudes, PhD",Wed Dec 01 14:02:30 +0000 2021,Causes de décès selon le sexe : les Statistiqu...
2,"Richard Eudes, PhD",Wed Dec 01 14:02:28 +0000 2021,Causes de décès des jeunes et des enfants : le...
3,"Richard Eudes, PhD",Wed Dec 01 14:02:28 +0000 2021,Causes de décès des personnes âgées : les Stat...
4,Floriane Very,Wed Dec 01 13:00:52 +0000 2021,RT @Maireinfo2: La #population française pourr...
5,Maire info,Wed Dec 01 11:11:58 +0000 2021,La #population française pourrait baisser à pa...
6,SPI Toulouse Les Minimes,Wed Dec 01 10:00:20 +0000 2021,Retrouvez les chiffres clés de la dernière enq...
7,"Richard Eudes, PhD",Wed Dec 01 09:07:30 +0000 2021,Capacité d'accueil des personnes âgées selon l...
8,Mamatt,Wed Dec 01 08:03:53 +0000 2021,RT @philippeherlin: L'#INSEE affirme que le #P...
9,cWilly,Wed Dec 01 07:49:18 +0000 2021,#Insee confirme la baisse démographique en Fra...


In [16]:
# we can also get all the fields of a tweet
def tweets_json(tweets):
    tweet_json=[]
    for tweet in tweets:
        tweet_json.append(tweet._json)
    return tweet_json
    

# Here we use json normalize to convert json file to a pandas dataframe
df_all_atts = pd.json_normalize(tweets_json(tweets))
df_all_atts.head()

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.lang,retweeted_status.entities.media,retweeted_status.extended_entities.media
0,Tue Nov 30 12:55:51 +0000 2021,1465665879191048193,1465665879191048193,L'#INSEE affirme que le #PouvoirDAchat des Fra...,True,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wed Dec 01 14:02:30 +0000 2021,1466045040912908295,1466045040912908295,Causes de décès selon le sexe : les Statistiqu...,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wed Dec 01 14:02:28 +0000 2021,1466045035754016779,1466045035754016779,Causes de décès des jeunes et des enfants : le...,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wed Dec 01 14:02:28 +0000 2021,1466045031794585605,1466045031794585605,Causes de décès des personnes âgées : les Stat...,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wed Dec 01 13:00:52 +0000 2021,1466029530381000708,1466029530381000708,RT @Maireinfo2: La #population française pourr...,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 5. Write data frame to s3

We have the data frame, now we want to save the data frame on s3. We want to save the data frame in format parquet. Because it has an integrated schema.

### 5.1 Configure s3 connection

Here we will set the s3 credential and the output path of the parquet file. As we will generate a parquet file each day. We would like to have the generation date inside the file name.


In [78]:
endpoint = os.environ['AWS_S3_ENDPOINT']
bucket = "pengfei"
current_date=date.today().strftime("%d-%m-%Y")
output_path = f"diffusion/demo_prod/tweet_{current_date}"


### 5.2 Write df to s3 as parquet file

In [79]:
# This function write a pandas dataframe to s3 in parquet format
def write_df_to_s3(df, endpoint, bucket_name, path):
    # Convert pandas df to Arrow table
    table = pa.Table.from_pandas(df)
    url = f"https://{endpoint}"
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': url})
    file_uri = f"{bucket_name}/{path}"
    pq.write_to_dataset(table, root_path=file_uri, filesystem=fs)

In [80]:
write_df_to_s3(df,endpoint,bucket,output_path)

## 6. Test the output parquet file

In [81]:
# This function read a parquet file and return a arrow table
def read_parquet_from_s3(endpoint: str, bucket_name, path):
    url = f"https://{endpoint}"
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': url})
    file_uri = f"{bucket_name}/{path}"
    str_info = fs.info(file_uri)
    print(f"input file metadata: {str_info}")
    dataset = pq.ParquetDataset(file_uri, filesystem=fs)
    table = dataset.read()
    return table

In [84]:
arrow_table=read_parquet_from_s3(endpoint,bucket,output_path)

# Convert back to pandas
df_new = arrow_table.to_pandas()
df_new.head()

input file metadata: {'name': 'pengfei/diffusion/demo_prod/tweet_24-11-2021', 'size': 0, 'type': 'directory'}


,name,date,text
0,V. RICHES-FLORES,Wed Nov 24 13:38:12 +0000 2021,https://t.co/2nt1wV9P6U\nRFR - L’inflation arr...
1,Drecfire 🥕🥕🥕,Wed Nov 24 12:57:15 +0000 2021,RT @blaisegrenier: Nous ferons un point d'ici ...
2,Takeo,Wed Nov 24 11:30:40 +0000 2021,Nous ferons un point d'ici quelques temps en c...
3,Thierry Bès,Wed Nov 24 10:16:40 +0000 2021,"RT @RichardEudes: En novembre 2021, le climat ..."
4,Ellisphere,Wed Nov 24 08:12:08 +0000 2021,"RT @RichardEudes: En novembre 2021, le climat ..."
